#### Importing packages

In [ ]:
# Load python packages

import copy
import json
import jsonlines
import krippendorff

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pymongo
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm

from collections import Counter
from datetime import datetime
from scipy.stats import chi2_contingency, kendalltau, pearsonr, spearmanr
from statsmodels.formula.api import ols
from statsmodels.stats.inter_rater import fleiss_kappa


In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("test")


## Overview 
This notebook calculates the worker agreement scores and correlations. 


1. <strong>Load annotations for a task</strong> 

2. <strong>Run analysis for agreement among workers</strong> 

3. <strong>Run analysis for correlations among collected data</strong> 


Set the following variables first:


In [ ]:
# Set one of the following options: "table_annotation", "adjusted_claim_annotation"
task_type = ""

# Load annotations into pd.Dataframe 'df' 
df = pd.read_pickle("")

# Set path to PubHealthTab dataset (dataset.jsonl) 
path_pubhealthtab = ""


-------------------------------

#### Preprocess dataset

In [ ]:
cols = list(df.columns)
cols.remove("language")
cols.remove("type")
cols.remove("timestamp")
cols.remove("answers")
cols.extend(['worker_id', 'outputs', 'times', 'events', 'feedback'])

annotations_df = pd.DataFrame(columns = cols)
index = 0

for i, row in df.iterrows(): 
    
    for worker_answer in row["answers"]:
        row["worker_id"] = worker_answer["worker_id"]
        annotations_df.at[index, "_id"] = row["_id"]
        annotations_df.at[index, "batch_id"] = row["batch_id"]
        annotations_df.at[index, "references"] = row["references"]
        annotations_df.at[index, "taskSet_id"] = row["taskSet_id"]
        annotations_df.at[index, "hit"] = row["hit"]
        
        annotations_df.at[index, "worker_id"] = worker_answer["worker_id"]
        annotations_df.at[index, "outputs"] = worker_answer["values"]["outputs"]
        annotations_df.at[index, "times"] = worker_answer["values"]["times"]
        annotations_df.at[index, "events"] = worker_answer["values"]["events"]
        annotations_df.at[index, "feedback"] = worker_answer["values"]["feedback"]
        index += 1

print(len(annotations_df))
annotations_df.head(3)


In [ ]:
output_df = pd.DataFrame(columns=["HIT_id", "batch_id", "taskSet_id", "worker_id", "claim_id", "claim", "table",
                                      "label", "header", "events"])

index = 0
for i, row in annotations_df.iterrows():
    for j in range(len(row["references"])):
        if row["references"][j]["g_id"]!=-1:
            continue

        output_df.at[index, "HIT_id"] = row["_id"]
        output_df.at[index, "batch_id"] = row["batch_id"]
        output_df.at[index, "taskSet_id"] = row["taskSet_id"]
        output_df.at[index, "worker_id"] = row["worker_id"]

        output_df.at[index, "claim_id"] = row["references"][j]["claim_db_id"]
        output_df.at[index, "claim"] = row["references"][j]["claim"]
        output_df.at[index, "table"] = row["references"][j]["table"]

        output_df.at[index, "label"] = row["outputs"][j]["label"]
        output_df.at[index, "header"] = row["outputs"][j]["header"]
        output_df.at[index, "events"] = row["events"][j]
        index += 1

print(len(output_df))


In [ ]:
output_df.head(3)

## Agreement scores

Agreement scores: 
* Krippendorf's alpha: works with nominal, ordinal, and interval data by
* Fleiss' kappa: categorical data
* Randolph's kappa: also categorical data; BUT to avoid the "high agreement, low kappa paradox" [2], Fleiss' kappa is known to be prone to when the true class distribution of the data is unbalanced [1]


Other agreement scores: 
* Scott's π => is equivalent to Fleiss' Kappa but for more than two judges 


In [ ]:
def reliability_matrix_for_kripp_alpha(df: pd.DataFrame, three_class_annotation = True):
    """Creates reliability matrix for calculation of Krippendorf's alpha"""
    if three_class_annotation: 
        df['label'] = df['label'].apply(lambda x: 2 if x == 3 else x)
    
    df = df[['worker_id', 'claim_id', 'label']].groupby(['worker_id', 'claim_id']).agg(np.max).reset_index()
    df = df.pivot(index = 'worker_id', columns = 'claim_id', values = 'label').fillna(np.nan)

    return df


def reliability_matrix_for_fleiss_kappa(df: pd.DataFrame, three_class_annotation = True):
    """Creates reliability matrix for calculation of Fleiss kappa"""
    if three_class_annotation: 
        df['label'] = df['label'].apply(lambda x: 2 if x == 3 else x)
    
    df = df[['claim_id', 'label']]
    df['count'] = 1
    df = df.groupby(['claim_id', 'label']).sum().reset_index()
    df = df.pivot(index = 'claim_id', columns = 'label', values = 'count').fillna(0)
    df = df[df.apply(lambda x : sum(x) == 3.0, axis=1)]
    
    return df


### F-Kappa

##### TASK 1

In [ ]:
fleiss_df = reliability_matrix_for_fleiss_kappa(output_df.copy(), three_class_annotation = False)

fleiss_kappa_val = fleiss_kappa(fleiss_df.values, method = 'fleiss')
print('Fleiss\' kappa is {}.'.format(fleiss_kappa_val))


##### TASK 3

In [ ]:
fleiss_df = reliability_matrix_for_fleiss_kappa(output_df.copy(), three_class_annotation = False)

fleiss_kappa_val = fleiss_kappa(fleiss_df.values, method = 'fleiss')
print('Fleiss\' kappa is {}.'.format(fleiss_kappa_val))


### R-Kappa

Arguments for Randolph's kappa additionally to Fleiss lappa: 
* Avoid the high agreement, low kappa paradox [2]
* I.e. a high value of observed agreement p, can be drastically lowered by a substantial imbalance of classes in the dataset
* Although raters have a high agreement => can result in low Fleiss kappa 
* Fleiss kappa makes assumptions about the distribution of classes => problematic if imbalance given [2]


##### TASK 1

In [ ]:
randolph_df = reliability_matrix_for_fleiss_kappa(output_df.copy(), three_class_annotation = False)

randolph_kappa_val = fleiss_kappa(randolph_df.values, method = 'randolph')
print('Randolph\'s kappa is {}'.format(randolph_kappa_val))


##### TASK 3

In [ ]:
randolph_df = reliability_matrix_for_fleiss_kappa(output_df.copy(), three_class_annotation = False)

randolph_kappa_val = fleiss_kappa(randolph_df.values, method = 'randolph')
print('Randolph\'s kappa is {}'.format(randolph_kappa_val))


### K-Alpha [4]
* Perfect agreement if K-alpha = 1
* Alpha = 0 if observed disagreement is equal to disagreement which would result if labels are chosen randomly 
* K-alpha applicable to: 
 - Any number of observers, not just two
 - Any number of categories, scale values, or measures
 - Any metric or level of measurement (nominal, ordinal, interval, ratio, and more)
 - Incomplete or missing data
 - Large and small sample sizes alike, not requiring a minimum
 
Arguments for Krippendorf's alpha additionally to Fleiss Kappa: 
* Can handle missing/incomplete data!
* Can handle dataset of different size


##### TASK 1

In [ ]:
kripp_df = reliability_matrix_for_kripp_alpha(output_df.copy(), three_class_annotation = False)
kalpha = krippendorff.alpha(kripp_df.values, level_of_measurement='nominal')
print('Krippendorff\'s alpha  {}'.format(kalpha))

kripp_df.head(3)


##### TASK 3

In [ ]:
kripp_df = reliability_matrix_for_kripp_alpha(output_df.copy(), three_class_annotation = False)
kalpha = krippendorff.alpha(kripp_df.values, level_of_measurement='nominal')
print('Krippendorff\'s alpha  {}'.format(kalpha))

kripp_df.head(3)


## Correlation 

"Correlation" measures used should depend on the type of variables being investigated:
* continuous variable v continuous variable: use "traditional" correlation - e.g. Spearman's rank correlation or Pearson's linear correlation.
* continuous variable v categorical variable: use an ANOVA F-test / difference of means
* categorical variable v categorical variable: use Chi-square / Cramer's V


### Correlation discrete variables
* Pair-wise correlation: Pearson's r, Kendall's τ, or Spearman's \rho 


In [ ]:
# Load final dataset

dataset = []
with jsonlines.open(path_pubhealthtab) as reader:
    for line in reader: 
        dataset.append(line)
    
print(f"{len(dataset)} total entries in dataset.")

# convert dataset into pd.DataFrame
dataset_df = pd.DataFrame(dataset)
dataset_df.head(3)


In [ ]:
corr_df = pd.DataFrame(columns = ["claim_len", "table_len"])
# corr_df = pd.DataFrame(columns = ["claim_len", "header_rationale_len", "table_len"])

corr_df['claim_len'] = [len(nltk.word_tokenize(x)) for x in dataset_df['claim']]
# corr_df['header_rationale_len'] = [len(x) for x in dataset_df['header_rationale']]
corr_df['table_len'] = [len(x["rows"]) for x in dataset_df['table']]

corr_df.head(3)


#### Pearson

In [ ]:
# Correlation matrix

correlation_mat = corr_df.corr() # default method = pearson's
sns.heatmap(correlation_mat, annot = True)
plt.show()


In [ ]:
# Test for significance 

for col in list(corr_df.columns):
    p_val = round(pearsonr(corr_df["claim_len"], corr_df[col])[1], 3)
    
    if p_val < 0.05: 
        print(f"The correlation coeff. between 'claim_len' and '{col}' is stat. significant (p-value = {p_val}).")
    else: 
        print(f"The correlation coeff. between 'claim_len' and '{col}' is NOT stat. significant (p-value = {p_val}).")
    

#### Kendall

In [ ]:
correlation_mat = corr_df.corr(method="kendall")
sns.heatmap(correlation_mat, annot = True)
plt.show()


In [ ]:
# Test for significance 

for col in list(corr_df.columns):
    p_val = round(kendalltau(corr_df["claim_len"], corr_df[col])[1], 2)
    
    if p_val < 0.05: 
        print(f"The correlation coeff. between 'claim_len' and '{col}' is stat. significant (p-value = {p_val}).")
    else: 
        print(f"The correlation coeff. between 'claim_len' and '{col}' is NOT stat. significant (p-value = {p_val}).")


#### Spearman

In [ ]:
correlation_mat = corr_df.corr(method="spearman")
sns.heatmap(correlation_mat, annot = True)
plt.show()


In [ ]:
# Test for significance 

for col in list(corr_df.columns):
    p_val = round(spearmanr(corr_df["claim_len"], corr_df[col])[1], 2)
    
    if p_val < 0.05: 
        print(f"The correlation coeff. between 'claim_len' and '{col}' is stat. significant (p-value = {p_val}).")
    else: 
        print(f"The correlation coeff. between 'claim_len' and '{col}' is NOT stat. significant (p-value = {p_val}).")


### Correlation categorical variables
* Chi-square test (2 categorical variables)


In [ ]:
corr_df['label'] = dataset_df["label"]
corr_df['has_table_caption'] = [1 if x["caption"] else 0 for x in dataset_df['table']]
corr_df['has_table_header'] = [1 if (x["header_horizontal"] and len(x["header_horizontal"])>0) or (x["header_vertical"] and len(x["header_vertical"])>0) 
                               else 0 for x in dataset_df['table']]

corr_df.head(3)


#### Chi-square test

In [ ]:
# label and has_caption

cont_table = pd.crosstab(corr_df["label"], corr_df["has_table_caption"]) 
print(chi2_contingency(cont_table)[1])
cont_table


In [ ]:
# label and has_header

cont_table = pd.crosstab(corr_df["label"], corr_df["has_table_header"]) 
print(chi2_contingency(cont_table)[1])
cont_table


### Correlation categorical (e.g. label) and discrete variables

* Using __ANOVA F-test__ (1 continuous and 1 categorical variable)

* <font color=blue>__Null-hypothesis__</font>: label values (SUPPORTS, REFUTES, NEI) is equally distributed across the 2nd variable, e.g. claim length

* If <font color=blue>p-value is less 0.05</font>, we reject the null-hypothesis and can say there is a __stat. significant relation__ between label and 2nd variable [5]

<br>


In [ ]:
model = ols('claim_len ~ label', data = corr_df).fit()
anova_result = sm.stats.anova_lm(model, typ=2)
print(f"P-value is {round(anova_result.iloc[0,3], 3)}")

corr_df[['label', 'claim_len']].boxplot(by='label')


In [ ]:
model = ols('table_len ~ label', data = corr_df).fit()
anova_result = sm.stats.anova_lm(model, typ=2)
print(f"P-value is {round(anova_result.iloc[0,3], 3)}")

corr_df[['label', 'table_len']].boxplot(by='label')


In [ ]:
corr_df.head(3)

In [ ]:
model = ols('claim_len ~ has_table_caption', data = corr_df).fit()
anova_result = sm.stats.anova_lm(model, typ=2)
print(f"P-value is {round(anova_result.iloc[0,3], 3)}")

corr_df[['has_table_caption', 'claim_len']].boxplot(by='has_table_caption')


In [ ]:
model = ols('claim_len ~ has_table_header', data = corr_df).fit()
anova_result = sm.stats.anova_lm(model, typ=2)
print(f"P-value is {round(anova_result.iloc[0,3], 3)}")

corr_df[['has_table_header', 'claim_len']].boxplot(by='has_table_header')


### References

    [1] https://files.eric.ed.gov/fulltext/ED490661.pdf
    
    [2] https://reader.elsevier.com/reader/sd/pii/089543569090158L?token=68830E1F9765B027D7AC8E0260BEF9640E96046B99C8C264BC3222EAB0FD1D41B9C7E24EC24E99C4003168D13B3B48DA&originRegion=eu-west-1&originCreation=20210718072311
    
    [3] http://up.csail.mit.edu/other-pubs/soylent.pdf
    
    [4] https://repository.upenn.edu/cgi/viewcontent.cgi?article=1043&context=asc_papers
    
    [5] https://support.minitab.com/en-us/minitab-express/1/help-and-how-to/modeling-statistics/anova/how-to/one-way-anova/interpret-the-results/key-results/
    
    